In [1]:
import os
import pandas as pd 

In [7]:
dataset_path = os.path.join(os.getcwd(),'..','dataset','Names.csv')

In [4]:
dataset = pd.read_csv(dataset_path)

In [5]:
from IPython.display import clear_output

In [102]:
unique_characters = set()
for indx,name in enumerate(dataset['Name']):
    try:
        for c in name:
            unique_characters.add(c)
    except:pass
unique_characters = {k:v for v,k in enumerate(unique_characters)}
unique_characters[' ']=len(unique_characters)

In [338]:
import torch

def char_to_index(character):
    indx = unique_characters.get(character,-1)
    assert indx!=-1 , "Value not in unique characters"
    return indx

def word_to_tensor(word):
    tensor = torch.zeros(len(word) , 1 , len(unique_characters))
    for line_indx,char in enumerate(word):
        tensor[line_indx][0][char_to_index(char)] = 1
    return tensor

def tensor_to_word(tensor):
    word = ''
     
    for v in tensor_to_index(tensor.detach()):
        word+=list(unique_characters.keys())[int(v)]
    return word

def tensor_to_index(tensor):
    return torch.argmax(tensor,dim=2)
    

### Dataset and Loader

In [315]:
from torch.utils.data import TensorDataset,DataLoader


### Model definition 

In [367]:
import torch.nn as nn

In [368]:
class RNN(nn.Module):
    
    def __init__(self , h_out_size):
        
        super(RNN,self).__init__()
        self.h_out_size = h_out_size
        
        self.h_out = nn.Linear(len(unique_characters)+h_out_size , h_out_size)
        self.output = nn.Sequential(
            nn.Linear(h_out_size, len(unique_characters)),
            nn.Softmax(dim=2)
        )
    
    def forward(self,input_tensor,h_prev):
        
        input_tensor = torch.cat((input_tensor,h_prev),dim=2)
        out_h = self.h_out(input_tensor)
        out_val = self.output(out_h)
        return out_h,out_val
        

In [369]:
#sample_input 
h_out_size_const = 64
t = word_to_tensor('sam')
h_start = torch.zeros(t.shape[0],t.shape[1],h_out_size_const)

In [458]:
rnn = RNN(h_out_size_const)
out_h,out_val = rnn(t,h_start)

### Model training 

In [459]:
Rnn = RNN(h_out_size_const)

In [460]:
optimizer = torch.optim.Adam(Rnn.parameters(),lr=0.05)

In [461]:
def train_word(word):
    tensor_word = word_to_tensor(word)
    h = torch.zeros(len(word),1,h_out_size_const)
    h,out_val = Rnn(tensor_word,h)
    return out_val,tensor_word

In [462]:
def find_loss(out_tensor,act_tensor):
    act_tensor = torch.cat((act_tensor[1:],word_to_tensor(' ')))
    loss = loss_fn(out_tensor.squeeze() , tensor_to_index(act_tensor).squeeze())
    return loss

In [463]:
find_loss(word_to_tensor('am '),word_to_tensor('sam'))

tensor(2.3575)

In [464]:
loss_fn = nn.CrossEntropyLoss()
loss_fn(torch.Tensor([[0.5,0.3,0.2]]),torch.Tensor([1]).long())

tensor(1.1398)

In [465]:
import matplotlib.pyplot as plt
import numpy as np

In [466]:
epochs = 5
loss_acc = []
for epoc in range(1, epochs + 1):

    for indx, name in enumerate(dataset['Name']):
        optimizer.zero_grad()
        if (type(name) != str): continue
        out, act_tensor = train_word(name)
        loss = find_loss(out, act_tensor)
        loss.backward()
        optimizer.step()
        loss_acc.append(loss.item())

    print('Completed epoch {} of {} with loss {}'.format(
        epoc, epochs, np.mean(np.array(loss_acc))))
    print(out)

Completed epoch 1 of 5 with loss 3.2192375113616043
tensor([[[2.0701e-24, 1.8453e-18, 7.3216e-25, 5.1106e-21, 8.6041e-27,
          2.6040e-21, 5.0477e-24, 9.3272e-22, 2.1269e-23, 1.0898e-23,
          5.8424e-24, 6.2778e-19, 2.9699e-30, 1.0000e+00, 2.6594e-22,
          1.0409e-24, 3.9180e-26, 1.4465e-27, 1.1167e-22, 2.6365e-21,
          3.6701e-29, 3.0439e-15, 7.9552e-30, 1.0363e-15, 1.7152e-25,
          2.9051e-21, 3.6215e-29]],

        [[2.0921e-28, 1.0770e-22, 1.4515e-29, 1.2298e-24, 5.3235e-33,
          1.5554e-27, 1.6453e-35, 2.4238e-24, 5.0955e-27, 1.0678e-25,
          1.5904e-26, 1.0000e+00, 6.5188e-36, 6.0183e-20, 5.2259e-24,
          1.2318e-23, 2.3952e-33, 9.3789e-35, 2.9496e-26, 2.0321e-24,
          1.8852e-38, 1.7135e-12, 1.1891e-39, 5.5071e-17, 1.9562e-29,
          2.9538e-26, 4.9131e-39]],

        [[9.1178e-19, 1.7509e-13, 4.1757e-19, 3.4519e-15, 8.4525e-21,
          1.9017e-16, 1.9396e-20, 1.4825e-15, 9.5468e-18, 6.8793e-18,
          2.4547e-17, 4.6174e-08, 

In [457]:
out,act_tensor = train_word(dataset['Name'][2])
out

tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]]], grad_fn=<SoftmaxBackward>)